In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize  
from torchtext.vocab import Vocab 

script = pd.read_excel('../data/레이블링된 대본.xlsx')
script

,회차,시간,인물,대사,형태소,대사 레이블,형태소 레이블
0,1,0~5,# 상황,비오는 밤길을 달려가는 차량 헤드라이트 클로즈업 및 차창 밖으로 비가 내리고 있는 ...,"['비오', '밤길', '차량', '및', '차창', '밖', '비', '내리', ...",unknown,unknown
1,1,0~5,# 상황,비가 떨어지는 자동차 바닥 아래로 스타워즈 오프닝 스타일의 자막 흘러감,"['비', '떨어지', '자동차', '바닥', '스타워즈', '오프닝', '스타일'...",unknown,unknown
2,1,0~5,나레이션,이 이야기는 암울했던 민족의 수난기와 격동기의 역사를 살다 갔던 영원한 야인 김두한...,"['암울', '하', '았', '던', '민족', '수난', '격동기', '역사',...",unknown,민족주의
3,1,0~5,나레이션,그러나 드라마의 원만한 진행을 위하여 시대와 역사적 상황을 운영하는 인물 일부분에서...,"['그러나', '하', 'ㄴ', '진행', '위하', '시대', '역사', '적',...",unknown,unknown
4,1,0~5,# 장소,어느 도예촌 전경,"['어느', '도예', '촌', '전경']",unknown,unknown
...,...,...,...,...,...,...,...
61619,124,60~65,# 사물,김두한 영정 클로즈 업,"['김두한', '영정', '업']",unknown,unknown
61620,124,60~65,나레이션,김두한 그는 일제 말 우리가 주권을 잃었던 식민지 시절부터 해방 이후 좌우익의 대립...,"['김두한', '그', '일제', '말', '우리', '주권', '잃', '었', ...",국가주의,국가주의
61621,124,60~65,최동열,나는 오랫동안 자네를 지켜보아 온 사람일세 자네는 자네답게 살았어 조선의 주먹 황제...,"['나', '오랫동안', '자네', '지켜보', '오', 'ㄴ', '사람', 'ㄹ'...",unknown,왕정주의
61622,124,60~65,최동열,나름대로 자네의 역사를 가지고 자네의 시대를 치열하고 열심히 살았다는 얘기야 뭐랄까...,"['나름', '자네', '역사', '고', '자네', '시대', '치열', '하',...",unknown,unknown


In [2]:
# 형태소 레이블에서 unknown인 행 삭제
script = script[script['형태소 레이블'] != 'unknown']
script

,회차,시간,인물,대사,형태소,대사 레이블,형태소 레이블
2,1,0~5,나레이션,이 이야기는 암울했던 민족의 수난기와 격동기의 역사를 살다 갔던 영원한 야인 김두한...,"['암울', '하', '았', '던', '민족', '수난', '격동기', '역사',...",unknown,민족주의
25,1,0~5,김두한,오늘 국회에서 마지막 발언이 있습니다 선생님 잠시만 나와보시겠습니까,"['오늘', '국회', '마지막', '발언', '있', '습니다', '선생님', '...",unknown,민주주의
27,1,0~5,최동열,국회에,['국회'],unknown,민주주의
37,1,5~10,공원관리인,여러분네들 아니 국회에서 여기서 똥 퍼다가 뭐한다요 아니 이 밤중에 도대체 뭘들 한다요,"['네', '들', '국회', '서', '똥', '퍼다', '뭐', '하', 'ㄴ'...",unknown,민주주의
40,1,5~10,정대발,쉬이 조용히 저 인분은 피흘려 나라를 지키신 애국지사 분들께서 기미년 삼월 일일날에...,"['조용히', '저', '피', '흘리', '어', '지키', '시', 'ㄴ', '...",민족주의,민족주의
...,...,...,...,...,...,...,...
61588,124,50~55,나레이션,김두한은 오물사건에서 병보석으로 석방된 이후 청와대에서 박정희를 만났다고 한다 그 ...,"['김두한', '오물', '사건', '병', '보석', '석방', '되', 'ㄴ',...",unknown,국가주의
61606,124,55~60,두한,용서해 주십시오 그게 다 애국을 하느라고 그렇게 됐습니다 용서해 주십시오 용서해 주...,"['용서', '하', '주', '시', 'ㅂ오', '그', '게', '다', '애국...",unknown,민족주의
61616,124,55~60,나레이션,천구백칠십이년 십일월 이십일일 김두한은 오랜 지병이었던 고혈압으로 쓰러졌다 향년 오...,"['천', '구백', '칠십', '년', '십', '일월', '일', 'ㄹ', '김...",국가주의,국가주의
61620,124,60~65,나레이션,김두한 그는 일제 말 우리가 주권을 잃었던 식민지 시절부터 해방 이후 좌우익의 대립...,"['김두한', '그', '일제', '말', '우리', '주권', '잃', '었', ...",국가주의,국가주의


In [3]:
import torch
import torch.nn as nn

# 한국어 토크나이저 임포트
from konlpy.tag import Komoran 

# 사용자 정의 vocab 클래스 정의
class Vocab:
    def __init__(self, tokens, min_freq=1):
        self.tokens = tokens
        self.token_freq = {token: tokens.count(token) for token in set(tokens)}
        self.token_to_idx = {token: idx for idx, token in enumerate([token for token, freq in self.token_freq.items() if freq >= min_freq])}
        self.idx_to_token = {idx: token for token, idx in self.token_to_idx.items()}
        
    def __len__(self):
        return len(self.token_to_idx)

In [4]:
from konlpy.tag import Komoran
import re
from soynlp.normalizer import emoticon_normalize

# 불용어 제거 함수
def remove_unnecessary_sounds(morpheme):
    # 이모티콘 정규화 and 웃음 소리나 의미없는 소리 제거
    cleaned_morpheme = emoticon_normalize(morpheme, num_repeats=2) if morpheme not in ['하하', '하하하', '흠', '음', '허허', '음하하', '흐흐흐', '흐', '흐흐', '으응', '허허허', '으', '으허허', '으허'] else ''
    return cleaned_morpheme

# 토크나이저
tokenizer = Komoran()

vocab_tokens = []
for _, row in script.iterrows():
    tokens = tokenizer.morphs(row['형태소'])
    cleaned_tokens = [remove_unnecessary_sounds(token) for token in tokens]
    vocab_tokens.extend(cleaned_tokens)

# unk 추가
vocab_tokens.append('<unk>')

vocab = Vocab(vocab_tokens)

In [62]:
# # 데이터 전처리 및 Dataset 생성
# texts = []
# labels = []
# for _, row in script.iterrows():
#     text = [vocab.token_to_idx.get(token, vocab.token_to_idx['<unk>']) for token in tokenizer.morphs(row['대사'])]
#     label = row['레이블']
#     texts.append(text)
#     labels.append(label)
#     
# # 데이터셋 분리
# train_texts = texts[:int(0.8*len(texts))]
# train_labels = labels[:int(0.8*len(labels))]
# test_texts = texts[int(0.8*len(texts)):]
# test_labels = labels[int(0.8*len(labels)):]

In [6]:
# Hyperparameters
embed_size = 64
hidden_size = 128
num_classes = 9  # 클래스 개수
num_epochs = 10
learning_rate = 0.001
batch_size = 32

In [7]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class ScriptDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

    def collate_fn(self, batch):
        texts, labels = zip(*batch)
        texts = pad_sequence([torch.tensor(text) for text in texts], batch_first=True)
        labels = torch.tensor(labels, dtype=torch.long)
        return texts, labels

In [8]:
from collections import Counter

# 레이블 매핑
label_map = {
    'unknown': 0, 
    '왕정주의': 1, 
    '민족주의': 2, 
    '사회주의': 3, 
    '자유주의': 4, 
    '반공주의': 5, 
    '민주주의': 6, 
    '국가주의': 7,
    '제국주의': 8
}

# 데이터 전처리 및 Dataset 생성
texts = []
labels = []
for _, row in script.iterrows():
    text = [vocab.token_to_idx.get(token, vocab.token_to_idx['<unk>']) for token in tokenizer.morphs(row['형태소'])]
    label = label_map[row['형태소 레이블']]
    texts.append(text)
    labels.append(label)

train_dataset = ScriptDataset(texts[:int(0.8*len(texts))], labels[:int(0.8*len(labels))])
test_dataset = ScriptDataset(texts[int(0.8*len(texts)):], labels[int(0.8*len(labels)):])

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_dataset.collate_fn)

In [9]:
# RNN 모델 정의
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = self.fc(x[:, -1, :])
        return x

# 모델, 손실함수, 옵티마이저 초기화
model = RNNModel(len(vocab), embed_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# 학습 루프
for epoch in range(num_epochs):
    for texts, labels in train_loader:
        outputs = model(texts)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {100 * (outputs.argmax(1) == labels).sum().item() / batch_size:.2f}%')

Epoch 1/10, Loss: 1.4832, Accuracy: 21.88%
Epoch 2/10, Loss: 1.7240, Accuracy: 9.38%
Epoch 3/10, Loss: 1.3931, Accuracy: 21.88%
Epoch 4/10, Loss: 1.4894, Accuracy: 21.88%
Epoch 5/10, Loss: 1.3934, Accuracy: 28.12%
Epoch 6/10, Loss: 1.4624, Accuracy: 21.88%
Epoch 7/10, Loss: 1.3672, Accuracy: 25.00%
Epoch 8/10, Loss: 1.1479, Accuracy: 28.12%
Epoch 9/10, Loss: 1.4522, Accuracy: 21.88%
Epoch 10/10, Loss: 1.6196, Accuracy: 9.38%


In [11]:
# 평가
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for texts, labels in test_loader:
        outputs = model(texts)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 45.33%


In [29]:
# Inverse label mapping
idx_to_label_map = {v: k for k, v in label_map.items()}

def predict_ideology(model, sentence):
    model.eval()
    tokenized = [vocab.token_to_idx.get(token, vocab.token_to_idx['<unk>']) for token in tokenizer.morphs(sentence)]
    tensor = torch.LongTensor(tokenized).unsqueeze(0)
    prediction = model(tensor)
    _, pred = torch.max(prediction, 1)
    return idx_to_label_map[pred.item()]

In [30]:
predict_ideology(model, '정말로 기다리시던 분들입니다! 본인 심영과 문외봉 동무를 소개하겠습니다!')

'민족주의'

In [33]:
# Hyperparameters
embed_size = 64
hidden_size = 32 # 줄임   
num_classes = 9  # 클래스 개수
num_epochs = 10
learning_rate = 0.0001 # 학습률 조정
batch_size = 64 # 배치 사이즈 증가

In [34]:
model2 = RNNModel(len(vocab), embed_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)

In [35]:
# 학습 루프
for epoch in range(num_epochs):
    for texts, labels in train_loader:
        outputs = model2(texts)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {100 * (outputs.argmax(1) == labels).sum().item() / batch_size:.2f}%')

Epoch 1/10, Loss: 1.5190, Accuracy: 12.50%
Epoch 2/10, Loss: 1.7043, Accuracy: 7.81%
Epoch 3/10, Loss: 1.4467, Accuracy: 7.81%
Epoch 4/10, Loss: 1.5693, Accuracy: 7.81%
Epoch 5/10, Loss: 1.3822, Accuracy: 15.62%
Epoch 6/10, Loss: 1.7382, Accuracy: 7.81%
Epoch 7/10, Loss: 1.5403, Accuracy: 9.38%
Epoch 8/10, Loss: 1.6934, Accuracy: 10.94%
Epoch 9/10, Loss: 1.3748, Accuracy: 12.50%
Epoch 10/10, Loss: 1.5048, Accuracy: 10.94%


In [46]:
# 평가
model2.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for texts, labels in test_loader:
        outputs = model2(texts)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 45.65%


In [45]:
predict_ideology(model, '정말 위대합니다 선생!')

'사회주의'

In [15]:
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes, num_layers, dropout_rate):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.gru(x)
        x = self.fc(x[:, -1, :])
        return x

In [47]:
# Hyperparameters
embed_size = 64
hidden_size = 32 # 줄임
num_classes = 9  # 클래스 개수
num_epochs = 10
learning_rate = 0.0001  # 조정된 학습률
batch_size = 64 # 늘림
num_layers = 2  # GRU 층 수
dropout_rate = 0.5  # 드롭아웃 비율

# 모델, 손실함수, 옵티마이저 초기화
modelG = GRUModel(len(vocab), embed_size, hidden_size, num_classes, num_layers, dropout_rate)
criterion = nn.CrossEntropyLoss()
# adamw로 변경
optimizer = torch.optim.AdamW(modelG.parameters(), lr=learning_rate)

In [48]:
# 학습 루프
for epoch in range(num_epochs):
    for texts, labels in train_loader:
        outputs = modelG(texts)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {100 * (outputs.argmax(1) == labels).sum().item() / batch_size:.2f}%')

Epoch 1/10, Loss: 1.8963, Accuracy: 7.81%
Epoch 2/10, Loss: 1.6433, Accuracy: 9.38%
Epoch 3/10, Loss: 1.5039, Accuracy: 9.38%
Epoch 4/10, Loss: 1.3531, Accuracy: 12.50%
Epoch 5/10, Loss: 2.2583, Accuracy: 9.38%
Epoch 6/10, Loss: 1.5728, Accuracy: 7.81%
Epoch 7/10, Loss: 1.4942, Accuracy: 9.38%
Epoch 8/10, Loss: 1.3544, Accuracy: 9.38%
Epoch 9/10, Loss: 1.6905, Accuracy: 7.81%
Epoch 10/10, Loss: 1.5202, Accuracy: 6.25%


In [49]:
# 평가
modelG.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for texts, labels in test_loader:
        outputs = modelG(texts)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 45.97%


In [50]:
predict_ideology(modelG, '우리는 투표 없이도 불의한 권력을 민주적으로 몰아낸 자랑스러운 역사를 기억합니다')

'사회주의'

In [51]:
# Hyperparameters
embed_size = 64
hidden_size = 32 # 줄임
num_classes = 9 
num_epochs = 100  # 에폭 수 증가 
learning_rate = 0.0001  # 조정된 학습률
batch_size = 64 # 늘림
num_layers = 2  # GRU 층 수
dropout_rate = 0.5  # 드롭아웃 비율

# 모델, 손실함수, 옵티마이저 초기화
modelR = GRUModel(len(vocab), embed_size, hidden_size, num_classes, num_layers, dropout_rate)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(modelR.parameters(), lr=learning_rate)

In [52]:
from torch.optim.lr_scheduler import StepLR

# 스케줄러 초기화
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

# 학습 루프
for epoch in range(num_epochs):
    for texts, labels in train_loader:
        outputs = modelR(texts)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 스케줄러 스텝 업데이트
    scheduler.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {100 * (outputs.argmax(1) == labels).sum().item() / batch_size:.2f}%, Learning rate: {scheduler.get_last_lr()[0]}')

Epoch 1/100, Loss: 1.8674, Accuracy: 7.81%, Learning rate: 0.0001
Epoch 2/100, Loss: 1.7727, Accuracy: 6.25%, Learning rate: 0.0001
Epoch 3/100, Loss: 1.7335, Accuracy: 7.81%, Learning rate: 0.0001
Epoch 4/100, Loss: 1.4117, Accuracy: 10.94%, Learning rate: 0.0001
Epoch 5/100, Loss: 1.5685, Accuracy: 7.81%, Learning rate: 0.0001
Epoch 6/100, Loss: 1.5706, Accuracy: 6.25%, Learning rate: 0.0001
Epoch 7/100, Loss: 2.1440, Accuracy: 4.69%, Learning rate: 0.0001
Epoch 8/100, Loss: 1.8836, Accuracy: 4.69%, Learning rate: 0.0001
Epoch 9/100, Loss: 1.8417, Accuracy: 6.25%, Learning rate: 0.0001
Epoch 10/100, Loss: 1.4877, Accuracy: 12.50%, Learning rate: 0.0001
Epoch 11/100, Loss: 1.4986, Accuracy: 7.81%, Learning rate: 0.0001
Epoch 12/100, Loss: 1.4388, Accuracy: 10.94%, Learning rate: 0.0001
Epoch 13/100, Loss: 1.4617, Accuracy: 12.50%, Learning rate: 0.0001
Epoch 14/100, Loss: 1.6387, Accuracy: 9.38%, Learning rate: 0.0001
Epoch 15/100, Loss: 1.5337, Accuracy: 12.50%, Learning rate: 0.0001

In [54]:
# 평가
modelR.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for texts, labels in test_loader:
        outputs = modelR(texts)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 45.89%


In [55]:
predict_ideology(modelR, '고종황제 폐하 만세!')

'사회주의'